<a href="https://colab.research.google.com/github/fredionix/OpenPriceStockPredictionLSTM/blob/master/LSTM_Model_Predict_Open_Stock_Price_IBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import semua library yang dibutuhkan
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Dense, Dropout
import io

Using TensorFlow backend.


In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ecabbec5cf074364891db8919da0e5dc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QrodoBOlILG9oF_e0VdL6jCzNTfbKWMxaJqrkO0CvwFq',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ecabbec5cf074364891db8919da0e5dc.get_object(Bucket='assignment1-donotdelete-pr-uzd1pjjuaei5zu',Key='AAPL.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()



ModuleNotFoundError: ignored

In [0]:
print("checking if any null values are present\n", dataset.isna().sum())


dataset = dataset.drop(dataset.index[0]) #drop first index row, start with row 1, for easy split
#print(dataset)#print un droped dataset

#print("data set index 0 ", dataset.index[0])
dataset = dataset.drop(['Date'], axis=1) #drop date collumn
#print(dataset)#print un droped dataset


#print(dataset)#print un droped dataset
target = dataset.iloc[:,0]#select all rows and first collumn as target, open collumn as target for y test 33 and ytrain 66 
target =target.values#get values of open collumn

#print(target)
data = dataset.iloc[:,1:] #select all rows, from index 1 column to last collumn index, eliminate open col as target
data=data.values #getthe values of each row put it in the list
data = data.astype('float32')#declare float32 as type

scaler = MinMaxScaler(feature_range=(0, 1)) #create to scaling from 0-1 object range
data = scaler.fit_transform(data)#transform the values to 0-1 only

#load
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,train_size=0.8, random_state=42) #split to 33% test size, data go to x, test go to y, random state for initial shufling in the split process

#print bentuk xtrain ytrain xtest ytest

print("Please show me the shape of X_train:",X_train.shape)
print("Please show me the shape of X_test:",X_test.shape)
print("Please show me the shape of y_train:",y_train.shape)
print("Please show me the shape of y_test:",y_test.shape)

In [0]:
#reshape with format: row, 1, cols with train is 66% of row, in this case 25
X_train = np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))
#reshape with format: row, 1, cols with train is 66% of row, in this case 83
X_test = np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))
print("Please show me the shape of X_test:",X_test.shape)
print("Please show me the shape of X_train:",X_train.shape)

In [0]:
model = Sequential()
model.add(LSTM(125, input_shape=(1,5))) #input shape same the structure we reshaping
model.add(Dense(1)) #ini output layer
#ini continuous data bukan discrete, jadi algo lossnya men_squared error
model.compile(loss='mean_squared_error', optimizer='sgd')
#data is trained here

In [0]:
model.fit(X_train, y_train, epochs=1000, batch_size=10, verbose=1) #set verbose to 1 to keep up log, use 1000 epoch for 100 iteration training model
#prediction using X_test
Predict = model.predict(X_test) #predict x test for open data col
testScore = math.sqrt(mean_squared_error(y_test, Predict))#calculate test score


In [0]:
from matplotlib import pyplot
print('Test Score: %.2f RMSE' % (testScore))


plt.plot(y_test)#original value that we split
plt.plot(Predict)#prediction from xtest with the model keras
plt.legend(['original value','predicted value'],loc='upper right')
plt.show()

In [0]:
model.save('model.h5')#save the model
